In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## websraping using beautifulsoup

In [9]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html5lib')

In [7]:
tag_object=soup.title
print("tag object:",tag_object)

tag object: <title>List of postal codes of Canada: M - Wikipedia</title>


In [24]:
pip install geopy

In [26]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [10]:
toronto_data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

## Moving toronto data to dataframe

In [11]:
for row in soup.find("table").find_all("tr"):
    for col in row.find_all("td"):
        postalcode = col.p.b.text
        
        atag = (col.p).find_all("a")
        neighbor=""
        borough=""
        # print(len(atag))
        # print("{}-->".format(postalcode))
        if(len(atag) == 0):
            if(len((col.p).find_all("i"))==0):
                # print("{}".format(col.p.span.text))
                borough = col.p.span.text
            else:
                # print("{}".format(col.p.i.text))
                borough = col.p.i.text
        else:
            borough = atag[0].string
           
            # print("{}=>".format(borough.text))
            # print(type(atag))
            for i in range(1,len(atag)):
                # neighbor.append(atag[i].text)
                if(i==len(atag)-1):
                    neighbor+=("{}".format(atag[i].text))
                    # print("{}".format(atag[i].text))
                else:
                    # print("{},".format(atag[i].text))
                    neighbor+=("{},".format(atag[i].text))
            # print(neighbor)
        toronto_data = toronto_data.append({"PostalCode":postalcode,"Borough":borough,"Neighborhood":neighbor}, ignore_index=True)
        

In [62]:
toronto_data.head(10) 

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
5,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Queen's Park,
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern,Rouge"


In [12]:
toronto_data.shape

(180, 3)

In [2]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [3]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postal_code = "M5G"
# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [4]:
# geo_data = pd.read_csv("Geospatial_Coordinates.csv")
geo_data["Postal Code"]

0      M1B
1      M1C
2      M1E
3      M1G
4      M1H
      ... 
98     M9N
99     M9P
100    M9R
101    M9V
102    M9W
Name: Postal Code, Length: 103, dtype: object

In [14]:
geo_data[toronto_data["PostalCode"]==geo_data["Postal Code"]]

ValueError: Can only compare identically-labeled Series objects

In [15]:
df = pd.DataFrame()

In [18]:
print(type(toronto_data["PostalCode"]))

<class 'pandas.core.series.Series'>


In [19]:
print(type(geo_data["Postal Code"]))

<class 'pandas.core.series.Series'>


In [ ]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup


source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html5lib')

postal_codes_dict = {} # initialize an empty dictionary to save the data in
for table_cell in soup.find_all('td'):
    try:
        postal_code = table_cell.p.b.text # get the postal code
        postal_code_investigate = table_cell.span.text
        neighborhoods_data = table_cell.span.text # get the rest of the data in the cell
        borough = neighborhoods_data.split('(')[0] # get the borough in the cell
        
        # if the cell is not assigned then ignore it
        if neighborhoods_data == 'Not assigned':
            neighborhoods = []
        # else process the data and add it to the dictionary
        else:
            postal_codes_dict[postal_code] = {}
            
            try:
                neighborhoods = neighborhoods_data.split('(')[1]
            
                # remove parantheses from neighborhoods string
                neighborhoods = neighborhoods.replace('(', ' ')
                neighborhoods = neighborhoods.replace(')', ' ')

                neighborhoods_names = neighborhoods.split('/')
                neighborhoods_clean = ', '.join([name.strip() for name in neighborhoods_names])
            except:
                borough = borough.strip('\n')
                neighborhoods_clean = borough
 
            # add borough and neighborhood to dictionary
            postal_codes_dict[postal_code]['borough'] = borough
            postal_codes_dict[postal_code]['neighborhoods'] = neighborhoods_clean
    except:
        pass
    
# create an empty dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data

# populate dataframe with data from dictionary
for ind, postal_code in enumerate(postal_codes_dict):
    borough = postal_codes_dict[postal_code]['borough']
    neighborhood = postal_codes_dict[postal_code]['neighborhoods']
    toronto_data = toronto_data.append({"PostalCode": postal_code, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

# print number of rows of dataframe
toronto_data.shape[0]